In [15]:
!pip install gymnasium
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np
import itertools

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

gamma = 0.99
BS = 32
buff = 50000
replay_memory_size = 1000
eps_start = 1.0
eps_end = 0.02
eps_decay = 1000
update_target_freq = 1000

env = gym.make("CartPole-v0")

state_space = env.observation_space.shape[0]
action_space = env.action_space.n

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [16]:
class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()

        self.layer1 = nn.Linear(state_space, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, action_space)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

    def act(self, state):
        state_t = torch.as_tensor(state, dtype = torch.float32)
        q_values = self(state_t.unsqueeze(0))

        max_q_ind = torch.argmax(q_values, dim = 1)[0]
        action = max_q_ind.detach().item()

        return action

In [17]:
replay_memory = deque(maxlen=buff)
rewards_array = deque([0.0], maxlen=100)

In [18]:
ep_reward = 0.0

online_net = DQN()
target_net = DQN()
target_net.load_state_dict(online_net.state_dict())

optimizer = optim.Adam(online_net.parameters())

In [19]:
state, _ = env.reset()
for _ in range(replay_memory_size):
  action = env.action_space.sample()

  new_state, reward, terminated, truncated, _ = env.step(action)
  done = terminated or truncated
  transition = (state, action, reward, done, new_state)
  replay_memory.append(transition)
  state = new_state

  if done:
    state, _ = env.reset()

In [ ]:
state, _ = env.reset()

for step in itertools.count():
  eps = np.interp(step, [0, eps_decay], [eps_start, eps_end])
  samp = random.random()

  if samp <= eps:
    action = env.action_space.sample()
  else:
    action = online_net.act(state)

  new_state, reward, done, _, _ = env.step(action)
  transition = (state, action, reward, done, new_state)
  replay_memory.append(transition)
  state = new_state

  ep_reward += reward


  if done:
    state, _ = env.reset()
    rewards_array.append(ep_reward)
    ep_reward = 0.0

  transitions = random.sample(replay_memory, BS)

  states = torch.as_tensor(np.asarray([t[0] for t in transitions]), dtype = torch.float32)
  actions = torch.as_tensor(np.asarray([t[1] for t in transitions]), dtype = torch.int64).unsqueeze(-1)
  rewards = torch.as_tensor(np.asarray([t[2] for t in transitions]), dtype = torch.float32).unsqueeze(-1)
  dones = torch.as_tensor(np.asarray([t[3] for t in transitions]), dtype = torch.float32).unsqueeze(-1)
  new_states = torch.as_tensor(np.asarray([t[4] for t in transitions]), dtype = torch.float32)

  target_q = target_net(new_states)
  max_target_q = target_q.max(dim=1, keepdim=True)[0]

  targets = rewards + gamma*(1-dones)*max_target_q

  q_values = online_net(states)
  action_q_values = torch.gather(input = q_values, dim=1, index = actions)
  loss = F.smooth_l1_loss(action_q_values, targets)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if step % update_target_freq ==0:
    target_net.load_state_dict(online_net.state_dict())

  if step%1000==0:
    print()
    print("Step", step)
    print("Avg Rew", np.mean(rewards_array))